## Scrape Wikipedia for all the movies about their information on box office and budget.
We can clean result for the budget, but the result for the box office is too difficult to clean. We will get box office information from other sources.

In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

from pyquery import PyQuery as pq
from bs4 import BeautifulSoup
# The "requests" library makes working with HTTP requests easier
# than the built-in urllib libraries.
import requests
import re
import string
import json
from tqdm import tqdm

Scrape wikipedia page of each Oscar into yearstext dictionary

In [2]:
SUFFIXES = {1: 'st', 2: 'nd', 3: 'rd'}
def ordinal(num):
    if 10 <= num % 100 <= 20:
        suffix = 'th'
    else:
        suffix = SUFFIXES.get(num % 10, 'th')
    return str(num) + suffix

In [3]:
yearstext={}
for year in range(1,88):
    wikipage="https://en.wikipedia.org/wiki/{}_Academy_Awards".format(ordinal(year))
    if requests.get(wikipage).status_code==404:
        print year
    else:
        yearstext[year] = requests.get(wikipage)
    

Now we create a function to get the movie urls from each Oscar wikipedia page.

In [5]:
#test
year=70
yeartextdict = yearstext[year]
soup = BeautifulSoup(yeartextdict.text, "html.parser")
rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
if year in range(9,59)+[69]:
    s1=1
    s2=3
    s3=5
else:
    s1=0
    s2=1
    s3=2
bp_text=rows[s1].find_all("td")[0]
bd_text=rows[s1].find_all("td")[1]
b_actor_text=rows[s2].find_all("td")[0]
b_actress_text=rows[s2].find_all("td")[1]
b_s_actor_text=rows[s3].find_all("td")[0]
b_s_actress_text=rows[s3].find_all("td")[1]
bp_text.find_all("li")[0].find_all("i")[0].find("a").get_text()


u'Titanic'

This function applies to each Oscar wikipedia page for the 9th to 59th Oscars. It returns the movie urls and the movie titles and a dictionary with keys as movie urls and value as movie titles.

In [6]:
#function to get movie url for the 9th to 59th Oscars
def get_movie_url_2(year, yearstext):
    yeartextdict = yearstext[year]
    soup = BeautifulSoup(yeartextdict.text, "html.parser")
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
    if year in range(9,59)+[69]:
        s1=1
        s2=3
        s3=5
    else:
        s1=0
        s2=1
        s3=2
    bp_text=rows[s1].find_all("td")[0]
    bd_text=rows[s1].find_all("td")[1]
    b_actor_text=rows[s2].find_all("td")[0]
    b_actress_text=rows[s2].find_all("td")[1]
    b_s_actor_text=rows[s3].find_all("td")[0]
    b_s_actress_text=rows[s3].find_all("td")[1]
    movie_url=[]
    movie_title=[]
    for m in bp_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in bd_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_actor_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_actress_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_s_actor_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    for m in b_s_actress_text.find_all("li")[0].find_all("i"):
        if m.find("a")==None:
            a=1
        else:
            movie_url.append(m.find("a").get("href"))
            movie_title.append(m.find("a").get_text())
    movies=dict(zip(movie_url,movie_title))
    return movie_url , movie_title, movies
    

This function applies to each Oscar wikipedia page for the 9th to 59th Oscars. It returns the movie urls and the movie titles and a dictionary with keys as movie urls and value as movie titles.

In [7]:
#function to get movie url for first 9 Oscars
def get_movie_url_1(year, yearstext):
    yeartextdict = yearstext[year]
    soup = BeautifulSoup(yeartextdict.text, "html.parser")
    rows = soup.find("table", attrs={"class": "wikitable"}).find_all("tr")[0:]
    if year ==1 :
        s1=0
        s2=1
        s3=2
        bp_text=rows[s1].find_all("td")[0]
        bd_text=rows[s2].find_all("td")[0]
        bd_text2=rows[s2].find_all("td")[1]
        b_actor_text=rows[s3].find_all("td")[0]
        b_actress_text=rows[s3].find_all("td")[1]
        #b_s_actor_text=rows[s3].find_all("td")[0]
        #b_s_actress_text=rows[s3].find_all("td")[1]
        movie_url=[]
        movie_title=[]
        for m in bp_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text2.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actor_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actress_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
                
    else :
        s1=1
        s2=3
        bp_text=rows[s1].find_all("td")[0]
        bd_text=rows[s1].find_all("td")[1]
        b_actor_text=rows[s2].find_all("td")[0]
        b_actress_text=rows[s2].find_all("td")[1]
        #b_s_actor_text=rows[s3].find_all("td")[0]
        #b_s_actress_text=rows[s3].find_all("td")[1]
        movie_url=[]
        movie_title=[]
        for m in bp_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in bd_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actor_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
        for m in b_actress_text.find_all("li")[0].find_all("i"):
            if m.find("a")==None:
                a=1
            else:
                movie_url.append(m.find("a").get("href"))
                movie_title.append(m.find("a").get_text())
    movies=dict(zip(movie_url,movie_title))
    return movie_url , movie_title, movies

Apply the function to all the wikipedia Oscar pages.

In [9]:
movie_url=[]
movie_titles=[]
movie_dict={} #movie_url as key -- movie title
for year in range(1,10):
    movie_url.extend(get_movie_url_1(year,yearstext)[0])
    movie_titles.extend(get_movie_url_1(year,yearstext)[1])
    movie_dict.update(get_movie_url_1(year,yearstext)[2])
for year in range(9,88):
    #print year
    movie_url.extend(get_movie_url_2(year,yearstext)[0])
    movie_titles.extend(get_movie_url_2(year,yearstext)[1])
    movie_dict.update(get_movie_url_2(year,yearstext)[2])

Check that the lenths are as desired.

In [10]:
movie_url=list(set(movie_url))
movie_titles=list(set(movie_url))
#movie_dict
len(movie_url),len(movie_titles),len(movie_dict)

(1275, 1275, 1275)

Scrape wikipedia to get all the movie wikipedia pages. These are stored into ```moviestex``` dictionary. In case of failed ```get.``` attempt, we store all the failed urls into the ```failed_url``` list. It turns out that all the ```get.``` attemps are successful.

In [ ]:
moviestext={}
failed_url=[]
for url in movie_url:
    wikipage="https://en.wikipedia.org{}".format(url)
    if requests.get(wikipage).status_code==404:
        failed_url.append(url)
    else:
        moviestext[url] = requests.get(wikipage)
        time.sleep=1.0

Exception in thread Thread-1:
Traceback (most recent call last):
  File "/Users/rachelfyy/anaconda/lib/python2.7/threading.py", line 810, in __bootstrap_inner
    self.run()
  File "/Users/rachelfyy/anaconda/lib/python2.7/site-packages/ipykernel/parentpoller.py", line 37, in run
    time.sleep(1.0)
TypeError: 'float' object is not callable



In [ ]:
len(moviestext),len(failed_url)

(1275, 0)

In [ ]:
movietextdict=moviestext[moviestext.keys()[0]]

We now write a function for each movie page. This function takes ```movie_url``` and ```moviestext``` (the dictionary containing all scraped text) and returns the budget and box office information in list forms.

In [ ]:
def get_budget(movie_url,moviestext):
    movietextdict=moviestext[movie_url]
    bad_movie=[]
    budget=["NA"]
    box_office=["NA"]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    if soup.find("table", attrs={"class": "infobox vevent"})==None:
        bad_movie.append(movie_url)
    else:
        rows = soup.find("table", attrs={"class": "infobox vevent"}).find_all("tr")[0:]
        for r in rows:
            if r.find("th")==None:
                continue
            if r.find("th").get_text()=="Budget":
                budget=[]
                budget=[r.find("td").get_text().split("[")[0]]

            if r.find("th")==None:
                continue
            if r.find("th").get_text()=="Box office":
                box_office=[]
                for b in r.find("td").get_text().split("\n"):
                    box_office.append(b.split("[")[0])
    
    return budget,box_office
        


In [ ]:
##test
get_budget(moviestext.keys()[0],moviestext)

([u'A$6 million'], [u'A$2,898,000 (Australia)', u'US$10,278,575'])

Apply the function and check the resulting lengths.

In [ ]:
movieurl_budget_dict={}
movieurl_bo_dict={}
for movieurl in moviestext.keys():
    #print movieurl
    movieurl_budget_dict[movieurl],movieurl_bo_dict[movieurl]=get_budget(movieurl,moviestext)


In [ ]:
#movieurl_bo_dict
len(movieurl_budget_dict),len(movieurl_bo_dict)

(1275, 1275)

Write a function takes in each movie page and return the list of imdb link ```imdb_link``` and list of rotten tomatoes link ```rt_link``` .

In [ ]:
movietextdict=moviestext[moviestext.keys()[0]]

In [ ]:
###test
'''soup = BeautifulSoup(movietextdict.text, "html.parser")
#rows=soup.find_parents("")[0:][-1].next_element.next_element
rows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]
#rows=soup.find_all("table")[0:] #,attrs={"class": "mbox-small plainlinks sistersitebox"}
external_links=[]
for row in rows:
    if row.find("i")!= None:
        external_links.append(row.get("href"))

for l in external_links:
    if "imdb" in l:
        imdb_link=l
    if "rottentomatoes" in l:
        rt_link=l
'''

'soup = BeautifulSoup(movietextdict.text, "html.parser")\n#rows=soup.find_parents("")[0:][-1].next_element.next_element\nrows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]\n#rows=soup.find_all("table")[0:] #,attrs={"class": "mbox-small plainlinks sistersitebox"}\nexternal_links=[]\nfor row in rows:\n    if row.find("i")!= None:\n        external_links.append(row.get("href"))\n\nfor l in external_links:\n    if "imdb" in l:\n        imdb_link=l\n    if "rottentomatoes" in l:\n        rt_link=l\n'

In [ ]:
#function to get imbd movie id and rotten tomatoes url
def get_imbd_rt_url(movie_url,moviestext):
    movietextdict=moviestext[movie_url]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    rows=soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]
    external_links=[]
    imdb_link="NA"
    rt_link="NA"
    for row in rows:
        if row.find("i")!= None:
            external_links.append(row.get("href"))

    for l in external_links:
        if "imdb" in l:
            imdb_link=l
        if "rottentomatoes" in l:
            rt_link=l
    return imdb_link,rt_link
    

In [ ]:
#test
get_imbd_rt_url(moviestext.keys()[0],moviestext)

(u'http://www.imdb.com/title/tt0086617/',
 u'http://www.rottentomatoes.com/m/year_of_living_dangerously/')

Apply the function to all movies.

In [ ]:
movieurl_imdb_link_dict={}
movieurl_rt_link_dict={}
for movieurl in moviestext.keys():
    #print movieurl
    movieurl_imdb_link_dict[movieurl],movieurl_rt_link_dict[movieurl]=get_imbd_rt_url(movieurl,moviestext)

In [ ]:
len(movieurl_imdb_link_dict),len(movieurl_rt_link_dict)

(1275, 1275)

Merge into dataframe.

In [ ]:
movie_url_df=pd.DataFrame(movie_dict.items(),columns=['movie_url', 'title'])
movie_budget_df=pd.DataFrame(movieurl_budget_dict.items(),columns=['movie_url', 'budget'])
movie_bo_df=pd.DataFrame(movieurl_bo_dict.items(),columns=['movie_url', 'box_office'])
movie_imdb_link_df=pd.DataFrame(movieurl_imdb_link_dict.items(),columns=['movie_url', 'imdb_link'])
movie_rt_link_df=pd.DataFrame(movieurl_rt_link_dict.items(),columns=['movie_url', 'rt_link'])
#movie_budget_df.head()
#movie_bo_df.head()
#movie_url_df.head()
#movie_imdb_link_df.head()

In [ ]:
frames = [movie_url_df, movie_budget_df, movie_bo_df]
df = pd.merge(movie_url_df, movie_budget_df, on='movie_url')
df=pd.merge(df, movie_bo_df, on='movie_url')
df=pd.merge(df, movie_imdb_link_df, on='movie_url')
df=pd.merge(df, movie_rt_link_df, on='movie_url')
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/


In [ ]:
print "movies without rt_link"  
print df[df["rt_link"]=="NA"].shape[1]

movies without rt_link
6


In [ ]:
df[df['rt_link']=="NA"].shape

(515, 6)

In [ ]:
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/


In [ ]:
#test
df['budget'][1262]
df['budget'][527][0]
df['budget'][527][0]

u'$33\u201340 million'

Check that all movies each has only one information in budget. It should be eigher the budget information or "NA" for lack of information on Wikipedia.

In [ ]:
for item in df['budget']:
    if len(item)!=1:
        print item

Cleaning process. 
There are several challenges because the format is very different. 

All are coded in unicode, but they have different format, for example,"\$40–50 million", "\$1.1 million","₤4 million" and so on. The unicode for "–" symble in "\$40–50 million" cannot be successfully extracted. Because there are only around 10 of these cases, we treate them as "NA".
 
The pound ₤ sign is also coded in unicode, but we found a way to take them out.
 
There are other different currencies, so we added a list named ```currency```.

If there are other errors in the process, we coded them as some form of "ERR".
Because there are relatively low numbers of these errors and they are difficult to fix, we also treat them as "NA".

In [ ]:
budget_sorted=[]
currency=[]
figure_error_1_index=[]
figure_error_2_index=[]
currency_error_index=[]
figure_error_index=[]
error=[]
x=[]
mm=0
for item in df['budget']:
    mm=mm+1
    #print mm-1
    for i in item:
        if mm in [278+1,344+1,403+1,427+1,487+1,527+1,892+1,896+1,990+1,1059+1]: #all those of the form $40–50 million
            currency.append("FERR1")
            budget_sorted.append('FERR1')
            #print i
            figure_error_1_index.append(mm-1)
            ###################################
        else:
            #print i
            if i=="NA": #takes care of the NA budgets
                budget_sorted.append("NA")
                currency.append("NA")
                
            else:
                if len(i.split("$"))==2: #all those of the form $1.1 million
                    j=[]
                    i=i.split("$")
                    j.append(i[0]) # currency marker
                    j=j+(i[1].split(" "))
                    #print j       # j has the form [u'', u'1,453,000'] or [u'', u'12', u'million']
                elif len(i.split("$"))==1: #all those of the form ₤4 million
                    ii=i.encode('ascii','ignore') #get the money figure in the "₤4 million"
                    j=["P"]
                    j=j+ii.split(" ")
                    #print j     # j has the form [u'P', u'1,453,000'] or [u'P', u'12', u'million']
                
                #takes care of the currency part
                if j[0]=='':
                    currency.append("USD")
                elif j[0]=="A":
                    currency.append("AUD")
                elif j[0]=="US" or j[0]=="(":
                    currency.append("USD")   
                elif j[0]=="P":
                    currency.append("GBP")  
                else:
                    currency_error_index.append(mm-1)
                    #print j,mm-1
                    currency.append("CERR")
                
                
                if len(j)==2:
                    budget_sorted.append(j[1].replace(",",""))
                #else:
                    #figure_error_2_index.append(mm-1)
                    #budget_sorted.append("FERR2") 
                elif 'million' in j:
                    unit=1000000
                    budget_sorted.append(int(float(j[1])*unit))
                else:
                    #print i
                    figure_error_index.append(mm-1)
                    budget_sorted.append("FERR")

In [ ]:
len(figure_error_1_index),len(figure_error_2_index),len(figure_error_index),len(currency_error_index)

(10, 0, 23, 11)

So in totle, we have 390+10+23+11=434 "NA"'s out of 1276 entries.

In [ ]:
len(budget_sorted),len(currency)

(1275, 1275)

In [ ]:
for m,b in enumerate(budget_sorted):
    if m not in [158,328,349,401,440,444,468,607,608,632,765,963,968,984,1052,1265]:
        if b not in ["NA","FERR","CERR","FERR1"]:
            '''#print m
            #print float(b)'''

In [ ]:
for i in [158,328,349,401,440,444,468,607,608,632,765,963,968,984,1052,1265]:
    #print budget_sorted[i]
    if isinstance(budget_sorted[i], (int, long)) == 0:
        budget_sorted[i]=budget_sorted[i].encode('ascii','ignore')
        if budget_sorted[i] in ["N/A","NA","Unidentified"]:
            budget_sorted[i]="NA"
        else:
            j=budget_sorted[i].split("m")
            budget_sorted[i]=int(float(j[0])*1000000)
    else:
        budget_sorted[i]=budget_sorted[i]

In [ ]:
for m,b in enumerate(budget_sorted):
    #print budget_sorted[m]
    if b in ["NA","FERR","CERR","FERR1", "Unknown"]:
        budget_sorted[m]="NA"
    else:
        budget_sorted[m]=float(budget_sorted[m])

for m,b in enumerate(currency):
    if b in ["NA","FERR","CERR","FERR1"]:
        currency[m]="NA"
    else:
        currency[m]=currency[m]
    

We merge them into dataset.

In [ ]:
df['budget_sorted']=budget_sorted
df['currency']=currency

Extract the imdb if for merging purpose later.

In [ ]:
imdb_id=[]
for l in df.imdb_link:
    if l!="NA":
        imdb_id.append(l.split("/")[-2])
    else:
        imdb_id.append("NA")

In [ ]:
df['imdb_id']=imdb_id
df.head()

,movie_url,title,budget,box_office,imdb_link,rt_link,budget_sorted,currency,imdb_id
0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...,6000000,AUD,tt0086617
1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NA,1750000,USD,tt0038053
2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...,NA,NA,tt041113
3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NA,NA,NA,tt0021310
4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/,7500000,USD,tt1174732


In [ ]:
df.to_csv('wiki_df_cleaned.csv', encoding='utf-8')

## Scrape IMDb to get the list of nominations and list of awards won by each movie. ##
This will be our results.

### Scrape IMDb for nomination information###

Check how many movies there are in total.

In [ ]:
df.shape[0]

1275

After scraping, we have 1265 movies, but that shouldn't be a problem, because we will delete the moview with run time less than 1 hour.

In [ ]:
movietext={}
for m_id in tqdm(df.imdb_id,leave=True):
    pg="http://www.imdb.com/title/{}/awards?ref_=tt_awd".format(m_id)
    movietext[m_id]=requests.get(pg)
    

len(movietext)

100%|██████████| 1275/1275 [06:56<00:00,  4.16it/s]


1265

In [ ]:
##test
m_id=movietext.keys()[20]
#test
m_id='tt0018389'
print m_id
movietextdict = movietext[m_id]
soup = BeautifulSoup(movietextdict.text, "html.parser")
rows = soup.find("table", attrs={"class":"awards"}).find_all("tr")[0:]
nominations=[]
wins=[]
won_q= "Won" in rows[0].find("td", attrs={"class":"title_award_outcome"}).get_text()
#print "Nominated" in rows[5].find("td", attrs={"class":"title_award_outcome"})


for i,r in enumerate(rows):
    #print i
    if r.find("td", attrs={"class":"title_award_outcome"})!=None:
        check= "Nominated" in r.find("td", attrs={"class":"title_award_outcome"}).get_text()
        if check==True:
            #print "t",i
            nn=i
        
#print nn

for i,r in enumerate(rows):
    print r
    rr=r.find_all("td", attrs={"class":"award_description"})[0:]
    if i<nn:
        for i in rr:
            if "Best Actor in a Leading Role" in i.get_text():
                wins.append('ML')
            if "Best Picture" in i.get_text():
                wins.append('BP')
            if "Best Actess in a Leading Role" in i.get_text():
                wins.append('FL')
            if "Best Actess in a Supporting Role" in i.get_text():
                wins.append('FS')
            if "Best Actor in a Supporting Role" in i.get_text():
                wins.append('MS')  
    else:
        for i in rr:
            if "Best Actor in a Leading Role" in i.get_text():
                nominations.append('ML')
            if "Best Picture" in i.get_text():
                nominations.append('BP')
            if "Best Actess in a Leading Role" in i.get_text():
                nominations.append('FL')
            if "Best Actess in a Supporting Role" in i.get_text():
                nominations.append('FS')
            if "Best Actor in a Supporting Role" in i.get_text():
                nominations.append('MS')   
nominations=nominations+wins
print nominations, wins

tt0018389
<tr>
<td class="title_award_outcome" rowspan="1">
<b>Nominated</b><br/>
<span class="award_category">Oscar</span>
</td>
<td class="award_description">
            Best Actress in a Leading Role<br/>
<a href="/name/nm0237571?ref_=ttawd_awd_1">Louise Dresser</a>
<br/>
</td>
</tr>
[] []


A function to get the nominations information. Will return a three things: movie id, list of nominations, list of wins

In [ ]:
def get_nomination(m_id,movietext):
    #print m_id
    movietextdict = movietext[m_id]
    soup = BeautifulSoup(movietextdict.text, "html.parser")
    nominations=[]
    wins=[]
    if soup.find("table", attrs={"class":"awards"})!=None:
        rows = soup.find("table", attrs={"class":"awards"}).find_all("tr")[0:]
        nn=0
        won_q= "Won" in rows[0].find("td", attrs={"class":"title_award_outcome"}).get_text()
        #print "Nominated" in rows[5].find("td", attrs={"class":"title_award_outcome"})


        for i,r in enumerate(rows):
            #print i
            if r.find("td", attrs={"class":"title_award_outcome"})!=None:
                check= "Nominated" in r.find("td", attrs={"class":"title_award_outcome"}).get_text()
                if check==True:
                    #print "t",i
                    nn=i

        #print nn

        for i,r in enumerate(rows):
            rr=r.find_all("td", attrs={"class":"award_description"})[0:]
            if i<nn:
                for ii in rr:
                    if "Best Actor in a Leading Role" in ii.get_text():
                        wins.append('ML')
                    if "Best Picture" in ii.get_text():
                        wins.append('BP')
                    if "Best Actress in a Leading Role" in ii.get_text():
                        wins.append('FL')
                    if "Best Actress in a Supporting Role" in ii.get_text():
                        wins.append('FS')
                    if "Best Actor in a Supporting Role" in ii.get_text():
                        wins.append('MS')  
            else:
                for ii in rr:
                    if "Best Actor in a Leading Role" in ii.get_text():
                        nominations.append('ML')
                    if "Best Picture" in ii.get_text():
                        nominations.append('BP')
                    if "Best Actress in a Leading Role" in ii.get_text():
                        nominations.append('FL')
                    if "Best Actress in a Supporting Role" in ii.get_text():
                        nominations.append('FS')
                    if "Best Actor in a Supporting Role" in ii.get_text():
                        nominations.append('MS')   
        nominations=nominations+wins
    
    return m_id, nominations, wins

In [ ]:
##test
m_id='tt0018389'
get_nomination(m_id,movietext)

('tt0018389', ['FL'], [])

We now merge into dataframe.

In [ ]:
bp_w=[]
ms_w=[]
fs_w=[]
ml_w=[]
fl_w=[]
bp_n=[]
ms_n=[]
fs_n=[]
ml_n=[]
fl_n=[]
movie_id=[]
for mid in movietext.keys():
    mid,n,w=get_nomination(mid,movietext)
    movie_id.append(mid)
    
    bp_w.append("BP" in w)
    ms_w.append("MS" in w)
    fs_w.append("FS" in w)
    ml_w.append("ML" in w)
    fl_w.append("FL" in w)
    
    bp_n.append("BP" in n)
    ms_n.append("MS" in n)
    fs_n.append("FS" in n)
    ml_n.append("ML" in n)
    fl_n.append("FL" in n)

print len(movie_id),len(bp_w),len(ms_w),len(fs_w),len(ml_w),len(fl_w),len(bp_n),len(ms_n),len(fs_n),len(ml_n),len(fl_n)
        

1265 1265 1265 1265 1265 1265 1265 1265 1265 1265 1265


In [ ]:
dict_for_df={}
dict_for_df['bp_w']=bp_w
dict_for_df['ms_w']=ms_w
dict_for_df['fs_w']=fs_w
dict_for_df['fl_w']=fl_w
dict_for_df['ml_w']=ml_w
dict_for_df['bp_n']=bp_n
dict_for_df['ms_n']=ms_n
dict_for_df['fs_n']=fs_n
dict_for_df['ml_n']=ml_n
dict_for_df['fl_n']=fl_n
dict_for_df['movie_id']=movie_id

dict_for_df.keys()

['ms_n',
 'bp_w',
 'fl_n',
 'fs_w',
 'ml_w',
 'ml_n',
 'movie_id',
 'ms_w',
 'fl_w',
 'bp_n',
 'fs_n']

In [ ]:
df1=pd.DataFrame(dict_for_df)

In [ ]:
df1.head()

,bp_n,bp_w,fl_n,fl_w,fs_n,fs_w,ml_n,ml_w,movie_id,ms_n,ms_w
0,False,False,False,False,False,False,True,False,tt0070666,False,False
1,False,False,False,False,False,False,False,False,tt2582802,False,False
2,False,False,False,False,True,False,False,False,tt0172493,False,False
3,True,False,False,False,False,False,True,False,tt0045943,False,False
4,False,False,True,False,False,False,True,False,tt0055895,False,False


In [ ]:
df1.to_csv('result.csv',  encoding='utf-8')

## Scrape wikipedia page for the list of movies released in 2015
 
We then scrapped [this wikipedia page](https://en.wikipedia.org/wiki/2015_in_film) to get the list of movies released in 2015. Because out scrapped in done in 4th of December, and the list of movies are updated upto that date, our list does not contain movies with release date after 4th of December. However, we consider the list of movies sufficient enough for us to make a prediction on.

In [ ]:
wikipage="https://en.wikipedia.org/wiki/2015_in_film"
yearstext= requests.get(wikipage)

In [ ]:
soup = BeautifulSoup(yearstext.text, "html.parser")
rows = soup.find_all("table", attrs={"class": "wikitable"})[0:]

In [ ]:
rows=rows[3:]

In [ ]:
r=rows[0]
m=r.find_all("tr")[0:][1]
m.find("a").get("href")

u'/wiki/Pinnokam'

Check the number of movies, and the length of the lists of information matches.

In [ ]:
movie_text=[]
for r in rows:
    movie_text.extend(r.find_all("tr")[1:])
len(movie_text)

317

In [ ]:
title=[]
url=[]
for m in movie_text:
    title.append(m.find("i").get_text())
    url.append(m.find("a").get("href"))
len(title),len(url)


# Save it to data frame.

(317, 317)

In [ ]:
dict_for_df={}
dict_for_df['titles_new']=title
dict_for_df['wiki_url_new']=url

In [ ]:
df_1=pd.DataFrame(dict_for_df)
print df_1.shape
df_1.head()

(317, 2)


,titles_new,wiki_url_new
0,Pinnokam,/wiki/Pinnokam
1,20 Once Again,/wiki/20_Once_Again
2,Blackhat,/wiki/Blackhat_(film)
3,I,/wiki/I_(film)
4,Wild Card,/wiki/Wild_Card_(2015_film)


In [ ]:
df_1.to_csv('new_movies.csv', encoding='utf-8')

In [ ]:
df = pd.read_csv('wiki_df_cleaned.csv')
df.head()

,Unnamed: 0,movie_url,title,budget,box_office,imdb_link,rt_link,budget_sorted,currency,imdb_id
0,0,/wiki/The_Year_of_Living_Dangerously_(film),The Year of Living Dangerously,[A$6 million],"[A$2,898,000 (Australia), US$10,278,575]",http://www.imdb.com/title/tt0086617/,http://www.rottentomatoes.com/m/year_of_living...,6000000,AUD,tt0086617
1,1,/wiki/Saratoga_Trunk,Saratoga Trunk,[$1.75 million],"[$4,250,000 (US/ Canada rentals) ]",http://www.imdb.com/title/tt0038053/,NaN,1750000,USD,tt0038053
2,2,/wiki/All_the_King%27s_Men_(1949_film),All the King's Men,[NA],[$2.4 million (US rentals)],http://www.imdb.com/title/tt041113/,http://www.rottentomatoes.com/m/1000654-all_th...,NaN,NaN,tt041113
3,3,/wiki/Romance_(1930_film),Romance,[NA],[NA],http://www.imdb.com/title/tt0021310/,NaN,NaN,NaN,tt0021310
4,4,/wiki/An_Education,An Education,[$7.5 million],"[$26,096,852]",http://www.imdb.com/title/tt1174732/,http://www.rottentomatoes.com/m/an_education/,7500000,USD,tt1174732


### Scraping Box Office from IMDb

We can find box office information of each movie on the IMDb website through url of the format http://www.imdb.com/title/tt0034167/business?ref_=tt_dt_bus. With the code below, we scrape the entire webpage for future use.

In [ ]:
box_office_page = []
imdb_id = []
bad_id = []
url = 'http://www.imdb.com/title/{}/business?ref_=tt_dt_bus'
for i in tqdm(df.index, leave = True):
    if requests.get(url.format(df.imdb_id[i])).status_code == 404:
        bad_id.append(df.imdb_id[i])
    else:
        box_office_page.append(requests.get(url.format(df.imdb_id[i])))
        imdb_id.append(df.imdb_id[i])

100%|██████████| 1275/1275 [14:50<00:00,  1.48it/s]


In [ ]:
(len(box_office_page), len(bad_id))

(1265, 10)

### Getting Gross Box Office
 
After scraping the entire webpage, we write a function ```get_gross_bo``` to find the gross box office of each movie. This function takes ```imdb_id```, and its ```box_office_page``` as input, and returns a dictionary that has all ```imdb_id```s as keys, and box offices as values.

In [ ]:
def get_gross_bo(imdb_id, box_office_page):
    gross = []
    dictionary = {}
    for i in tqdm(range(0,len(box_office_page)), leave = True):
        soup = BeautifulSoup(box_office_page[i].text, "html.parser")
        rows = soup.find('div', attrs={'class': 'business'}).find_all('h5')
        for r in rows:
            if r.get_text() == 'Gross':
                gross = r.next_element.next_element.encode('ascii','ignore').split('(')[0].split(' ')
        dictionary[imdb_id[i]] = gross
    return dictionary

In [ ]:
soup = BeautifulSoup(box_office_page[100].text, "html.parser")
rows = soup.find('div', attrs={'class': 'business'}).find_all('h5')
for r in rows:
    if r.get_text() == 'Gross':
        gross = r.next_element.next_element.encode('ascii','ignore').split('(')[0].split(' ')
        
gross

['\n$14,849,583', '']

In [ ]:
dictionary = get_gross_bo(imdb_id, box_office_page)

100%|██████████| 1265/1265 [01:23<00:00, 15.19it/s]


In [ ]:
df = pd.DataFrame(dictionary.items(), columns = ['imdb_id', 'box_office'])

In [ ]:
df.head()

,imdb_id,box_office
0,tt0070666,"[\n$27,274,150, ]"
1,tt2582802,"[\n$13,092,000, ]"
2,tt0172493,"[\n$28,871,190, ]"
3,tt0045943,"[\n$2,021,000, ]"
4,tt0055895,"[\n$17,800,000, ]"


In [ ]:
currency = []
val = []
for index,value in enumerate(df.box_office):
    if len(value) == 2:
        if '$' in value[0]:
            val.append(float(value[0].split("$")[1].replace(",","")))
            currency.append('USD')
        else:
            val.append(float(value[0].replace(",","").replace('\n','')))
            currency.append('USD')
    elif len(value) == 3:
        val.append(float(value[1].replace(",","")))
        currency.append(value[0].replace('\n',''))

In [ ]:
len(val),len(df),len(currency)

(1263, 1263, 1263)

In [ ]:
df['box_office'] = val
df['currency'] = currency
df.head()

,imdb_id,box_office,currency
0,tt0070666,27274150,USD
1,tt2582802,13092000,USD
2,tt0172493,28871190,USD
3,tt0045943,2021000,USD
4,tt0055895,17800000,USD


In [ ]:
df.to_csv('box_office.csv', encoding='utf-8')

### Scraping wiki info for movies in 2015

In [ ]:
new_movies = df_1

In [ ]:
new_movies.head()

,titles_new,wiki_url_new
0,Pinnokam,/wiki/Pinnokam
1,20 Once Again,/wiki/20_Once_Again
2,Blackhat,/wiki/Blackhat_(film)
3,I,/wiki/I_(film)
4,Wild Card,/wiki/Wild_Card_(2015_film)


In [ ]:
new_movies_page = []
bad_url = []
wikiurl = 'https://en.wikipedia.org{}'
for i in tqdm(new_movies.wiki_url_new, leave = True):
    url = wikiurl.format(i)
    if requests.get(url).status_code == 404:
        bad_url.append(url)
    else:
        new_movies_page.append(requests.get(url))

100%|██████████| 317/317 [01:08<00:00,  4.75it/s]


In [ ]:
def get_imbd_rt_url(movie_page):
    soup = BeautifulSoup(movie_page.text, "html.parser")
    rows = soup.find_all("a", attrs={"class": "external text", "rel":"nofollow"})[0:]
    external_links=[]
    imdb_link="NA"
    rt_link="NA"
    for row in rows:
        if row.find("i")!= None:
            external_links.append(row.get("href"))

    for l in external_links:
        if "imdb" in l:
            imdb_link=l
        if "rottentomatoes" in l:
            rt_link=l
    return imdb_link,rt_link

In [ ]:
imdb_link_new = []
rt_link_new = []
for i in tqdm(range(0,len(new_movies_page)),leave = True):
    imdb_link_new.append(get_imbd_rt_url(new_movies_page[i])[0])
    rt_link_new.append(get_imbd_rt_url(new_movies_page[i])[1])

100%|██████████| 317/317 [01:09<00:00,  5.00it/s]


In [ ]:
new_movies['imdb_link_new'] = imdb_link_new
new_movies['rt_link_new'] = rt_link_new

In [ ]:
imdb_id = []
for l in new_movies.imdb_link_new:
    if l != "NA":
        imdb_id.append(l.split("/")[-2])
    else:
        imdb_id.append("NA")

In [ ]:
new_movies['imdb_id'] = imdb_id

In [ ]:
new_movies.imdb_link_new[245]

u'http://www.imdb.com/title/tt3567288/'

### Scrape box office and budget

In [ ]:
box_office_page_new = []
bad_id = []
url = 'http://www.imdb.com/title/{}/business?ref_=tt_dt_bus'
for i in tqdm(new_movies.imdb_id, leave = True):
    if requests.get(url.format(i)).status_code == 404:
        bad_id.append(i)
        box_office_page_new.append('NA')
    else:
        box_office_page_new.append(requests.get(url.format(i)))

100%|██████████| 317/317 [04:04<00:00,  1.35it/s]


In [ ]:
def get_budget_bo(box_office_page):
    budget = []
    box_office = []
    soup = BeautifulSoup(box_office_page.text, "html.parser")
    rows = soup.find('div', attrs={'class': 'business'}).find_all('h5')
    for r in rows:
        if r.get_text() == 'Gross':
            box_office.append(r.next_element.next_element.encode('ascii','ignore').split('(')[0].split(' '))
        if r.get_text() == 'Budget':
            budget.append(r.next_element.next_element.encode('ascii','ignore').split('(')[0].split(' '))
    return budget, box_office

In [ ]:
budget_new = []
box_office_new = []
for i in tqdm(box_office_page_new, leave = True):
    if i != 'NA':
        budget_new.append(get_budget_bo(i)[0])
        box_office_new.append(get_budget_bo(i)[1])
    else:
        budget_new.append('NA')
        box_office_new.append('NA')

100%|██████████| 317/317 [00:33<00:00,  9.33it/s]


In [ ]:
new_movies['budget_new'] = budget_new
new_movies['box_office_new'] = box_office_new
new_movies.head()

,titles_new,wiki_url_new,imdb_link_new,rt_link_new,imdb_id,budget_new,currency_budget,box_office_new,currency_bo
0,Pinnokam,/wiki/Pinnokam,NA,NA,NA,NA,NA,NA,NA
1,20 Once Again,/wiki/20_Once_Again,http://www.imdb.com/title/tt4344878/,NA,tt4344878,NA,NA,234573,USD
2,Blackhat,/wiki/Blackhat_(film),http://www.imdb.com/title/tt2717822/,http://www.rottentomatoes.com/m/blackhat/,tt2717822,7e+07,USD,7097125,USD
3,I,/wiki/I_(film),http://www.imdb.com/title/tt2302966/,NA,tt2302966,3e+07,USD,NA,NA
4,Wild Card,/wiki/Wild_Card_(2015_film),http://www.imdb.com/title/tt2231253/,http://www.rottentomatoes.com/m/heat_2013/,tt2231253,3e+07,USD,NA,NA


In [ ]:
len(new_movies.box_office_new[5])

0

In [ ]:
def cleaner(var):
    currency = []
    val = []
    for i in var:
        if i != 'NA':
            if len(i) != 0:
                if len(i[0]) == 2:
                    if '$' in i[0][0]:
                        if 'CAD' not in i[0][0]:
                            val.append(float(i[0][0].split("$")[1].replace(",","")))
                            currency.append('USD')
                        else:
                            val.append(i[0][0].split('$')[1].split(' '))
                            currency.append('CAD')
                    else:
                        val.append(float(i[0][0].replace(",","").replace('\n','')))
                        currency.append('USD')
                elif len(i[0]) == 3:
                    val.append(float(i[0][1].replace(",","")))
                    currency.append(i[0][0].replace('\n',''))
            else:
                val.append('NA')
                currency.append('NA')
        else:
            val.append('NA')
            currency.append('NA')
    return currency, val

In [ ]:
currency_bo, box_office_new = cleaner(new_movies.box_office_new)
currency_budget, budget_new = cleaner(new_movies.budget_new)
len(box_office_new)

317

In [ ]:
new_movies['box_office_new'] = box_office_new
new_movies['currency_bo'] = currency_bo
new_movies['budget_new'] = budget_new
new_movies['currency_budget'] = currency_budget
new_movies = new_movies[['titles_new','wiki_url_new','imdb_link_new','rt_link_new','imdb_id','budget_new','currency_budget','box_office_new','currency_bo']]
new_movies.head()

,titles_new,wiki_url_new,imdb_link_new,rt_link_new,imdb_id,budget_new,currency_budget,box_office_new,currency_bo
0,Pinnokam,/wiki/Pinnokam,NA,NA,NA,NA,NA,NA,NA
1,20 Once Again,/wiki/20_Once_Again,http://www.imdb.com/title/tt4344878/,NA,tt4344878,NA,NA,234573,USD
2,Blackhat,/wiki/Blackhat_(film),http://www.imdb.com/title/tt2717822/,http://www.rottentomatoes.com/m/blackhat/,tt2717822,7e+07,USD,7097125,USD
3,I,/wiki/I_(film),http://www.imdb.com/title/tt2302966/,NA,tt2302966,3e+07,USD,NA,NA
4,Wild Card,/wiki/Wild_Card_(2015_film),http://www.imdb.com/title/tt2231253/,http://www.rottentomatoes.com/m/heat_2013/,tt2231253,3e+07,USD,NA,NA


In [ ]:
new_movies.to_csv('2015_movies_wiki.csv', encoding='utf-8')
